In [ ]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" spacy==3.1.1

In [ ]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" fastai==2.4.1

## Prepraring for inference

### Preparing the data for inference

In [ ]:
import glob
import numpy
from functools import partial
from fastcore.all import *
from fastai.text.all import *

In [ ]:
df_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
df_test.head(3)

In [ ]:
df_test.shape

Getting the list of ids. This would later be used to create the submission file

In [ ]:
ids = df_test.id.to_list()
ids[:2]

Getting the excerpts to a list. This makes it easier to make predictions and later to create the submission file.

In [ ]:
excerpts = df_test.excerpt.to_list()

## Loading the model

In [ ]:
bkwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/back_final_model')
fwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/forward_final_model')

In [ ]:
bkwd_model.predict(excerpts[0])[0][0]

# Predictions

In [ ]:
def ensamble(excerpts, bkwd_model, fwd_model):
    bkwd_preds = np.array([bkwd_model.predict(excerpt)[0][0] for excerpt in excerpts])
    fwd_preds = np.array([fwd_model.predict(excerpt)[0][0] for excerpt in excerpts])
    return np.mean([bkwd_preds, fwd_preds], axis=0)

In [ ]:
predictions = ensamble(excerpts, bkwd_model, fwd_model)
#predictions = [float("{0:.2f}".format(pred.item())) for pred in ensamble(bkwd_model, fwd_model, excerpts)]
#predictions = [float("{0:.2f}".format(model.predict(excerpt)[0][0])) for excerpt in excerpts]

# Creating the submission file

Creating a new dataframe

In [ ]:
submission_df = pd.DataFrame(list(zip(ids, predictions)), columns=['id', 'target'])

Saving the dataframe as csv for submission.

In [ ]:
submission_df.head(2)

In [ ]:
submission_df.to_csv('submission.csv', index=False)